In [2]:

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.lda_model
import numpy as np
from openai import OpenAI
import configparser 
import tiktoken
from difflib import SequenceMatcher
from datetime import datetime
import json
import ast
import csv
import os

SAVE_DIR = "Saved_files"
os.makedirs(SAVE_DIR, exist_ok=True)

In [3]:
# Download necessary NLTK data and handle stop words
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def get_custom_stop_words(search_keywords=None):
    
    # Get standard stopwords
    stop_words = set(stopwords.words('english'))
    
    # Words to keep (search keywords)
    words_to_keep = set()
    if search_keywords:
        # Convert keywords to lowercase and split multi-word terms
        for keyword in search_keywords:
            keyword = keyword.lower()
            # Add both the full phrase and individual words
            words_to_keep.add(keyword)
            for word in keyword.split():
                words_to_keep.add(word)
    
    # Remove search keywords from stopwords
    stop_words = stop_words - words_to_keep
    
    # Scientific paper terms to exclude (add to stopwords)
    scientific_terms = {
        # Citation terms
        'et', 'al', 'ref', 'reference', 'references', 'cited', 'cite',
        # Figure and table references
        'fig', 'figure', 'figures', 'table', 'tables', 'chart', 'charts',
        # Publication terms
        'published', 'journal', 'conference', 'proceedings',
        # Measurement units and numbers
        'vol', 'volume', 'pp', 'page', 'pages', 'doi'
    }
    
    # Add scientific terms to stopwords
    stop_words = stop_words.union(scientific_terms)
    
    return stop_words


[nltk_data] Downloading package punkt to C:\Users\STSI/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\STSI/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\STSI/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
# Some code for AI assistance
# Initialize the LLM API
def initialize_openai():
    config = configparser.ConfigParser()
    config.read('config_LLM.txt')
    api_key = config['LLM'].get('OPENAI_API_KEY')
    model_type = config['LLM'].get('MODEL_TYPE')
    return OpenAI(api_key=api_key), model_type

# Credit tracker to keep track of the cost when using paid API
class CreditTracker:
    def __init__(self):
        self.total_tokens = 0
        self.total_cost = 0
        self.cost_per_1k_tokens = 0.00015

    def update(self, tokens):
        self.total_tokens += tokens
        self.total_cost += (tokens / 1000) * self.cost_per_1k_tokens

    def get_stats(self):
        return {
            "total_tokens": self.total_tokens,
            "total_cost": round(self.total_cost, 4)
        }

credit_tracker = CreditTracker()

def num_tokens_from_string(string: str, model_name: str) -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [5]:
# Preprocess function
def preprocess_text(text, search_keywords):
    if not isinstance(text, (str, int, float)):
        return ''
    text = str(text)
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s-]', '', text)
    tokens = word_tokenize(text) # Split text into individual words/tokens using NLTK's tokenizer
    stop_words = get_custom_stop_words(search_keywords)
    tokens = [token for token in tokens if token not in stop_words] # Remove all stop words from our tokens using list comprehension
    lemmatizer = WordNetLemmatizer() # Initialize WordNet lemmatizer to reduce words to their base/dictionary form
    tokens = [lemmatizer.lemmatize(token) for token in tokens] # Convert each token to its lemma (e.g., "systems" → "system", "running" → "run")
    return ' '.join(tokens) # Combine all processed tokens back into a single string with spaces between words


# Function to convert string representation of list to actual list, replacing long strings with "Unknown"
def string_to_list(s):
    if isinstance(s, str):
        # Remove brackets and split by comma
        fields = [field.strip().strip("'") for field in s.strip('[]').split(',')]
        # Replace fields wit more than 100 characters with "Unknown"
        return ["Unknown" if len(field) > 100 else field for field in fields]
    return ["Unknown"]  # Return ["Unknown"] for empty or non-string entries

# Clean fields of study
def clean_fields_of_study(s):
    valid_fields= ['Computer Science', 'Economics', 'Engineering', 'Physics', 'Mathematics', 'Medicine', 'Business', 'Environmental Science', 'Chemistry', 'Materials Science', 'Geography', 'Biology', 'Geology', 'Political Science', 'Psychology', 'Com']
    if pd.isna(s) or s == '[]':
        return ["Unknown"]
    if isinstance(s, str):
        fields = [field.strip().strip("'\"") for field in s.strip('[]').split(',')]
        cleaned_fields = []
        for field in fields:
            if field in valid_fields:
                cleaned_fields.append(field)
            else:
                cleaned_fields.append("Unknown")
        return cleaned_fields if cleaned_fields else ["Unknown"]
    return ["Unknown"]

## Simple analysis to extract keyword and n-gram frequency

In [37]:
def get_term_frequencies(vectorizer, texts):
    matrix = vectorizer.fit_transform(texts)
    terms = vectorizer.get_feature_names_out()
    freqs = matrix.sum(axis=0).A1
    return dict(sorted(zip(terms, freqs.tolist()), key=lambda x: x[1], reverse=True))

def extract_keywords_and_ngrams(df, max_features=1000):
    # Create vectorizers
    keyword_vectorizer = CountVectorizer(
        max_df=0.95, 
        min_df=2,
        #stop_words='english',# can be removed as stopwords are already removed from processed_text
        max_features=max_features,
        token_pattern=r'(?u)\b[A-Za-z][A-Za-z-]+[A-Za-z]\b'
    )
    
    bigram_vectorizer = CountVectorizer(
        ngram_range=(2,2),
        max_df=0.95,
        min_df=2,
        #stop_words='english',# can be removed as stopwords are already removed from processed_text
        max_features=max_features
    )
    
    trigram_vectorizer = CountVectorizer(
        ngram_range=(3,3),
        max_df=0.95,
        min_df=2,
        max_features=max_features,
        #stop_words='english' # can be removed as stopwords are already removed from processed_text
    )
    
    # Extract frequencies
    keyword_freq = get_term_frequencies(keyword_vectorizer, df['processed_text'])
    bigram_freq = get_term_frequencies(bigram_vectorizer, df['processed_text'])
    trigram_freq = get_term_frequencies(trigram_vectorizer, df['processed_text'])
    
    # Save results
    current_date = datetime.now().strftime("%Y_%m_%d")
    filename=os.path.join(SAVE_DIR,f'term_frequencies_{current_date}.json')
    results = {
        'keywords': keyword_freq,
        'bigrams': bigram_freq,
        'trigrams': trigram_freq
    }
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    print(f"Results saved to {filename}")

In [39]:

def extract_topic_keywords(lda_model, vectorizer, num_words=10):
    """Extract keywords and n-grams for each topic"""
    feature_names = vectorizer.get_feature_names_out()
    topic_keywords = {}
    
    for topic_idx, topic in enumerate(lda_model.components_):
        # Get top words
        top_words = [feature_names[i] for i in topic.argsort()[:-num_words-1:-1]]
        
        # Get word weights
        word_weights = [(feature_names[i], topic[i]) 
                       for i in topic.argsort()[:-num_words-1:-1]]
        
        topic_keywords[topic_idx] = {
            'top_words': top_words,
            'word_weights': word_weights
        }
    
    return topic_keywords

# Model topics function
def model_topics_by_field(df, field, num_topics=10, num_words=5):
    df_field = df[df['fieldsOfStudy'].apply(lambda x: field in x)] # filtering so that only the documents within the field is keept.
    print (f"Analyzing {len(df_field)} papers")
    if df_field.empty:
            print(f"No papers found for field: {field}")            #warning if no papers found
            return None, None, None, None, None
    text_data = df_field['abstract'].fillna('')                     # filtering to avoid errors due to missing fields
    
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english') # vectorizing with max_df = 0.95 as default (terms that appear in more than 95% of documents) and min_df =2 (terms that appear in less than 2 of the documents). Values below 1 indicates percentile, and values above 1 indicates number of docs.
    doc_term_matrix = vectorizer.fit_transform(text_data) #creating the term matrix:  vectorizer : sklearn.feature_extraction.text.(CountVectorizer, TfIdfVectorizer).vectorizer used to convert raw documents to document-term matrix (`dtm`)
    
    #Using the sklearn decomposition LatentDirchletAllocation (see https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). 
    #Fit the LDA model
    lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42) 
    #Randomstate 42 is passed to maintain reproducibility in results (0 and 42 are commonly used values. Default "None" will result in using globa radnom instande (numpy.radom) and re-running may produce different results
    
    topic_distribution = lda_model.fit_transform(doc_term_matrix)
    topic_keywords=extract_topic_keywords(lda_model, vectorizer)
    
    #feature_names = vectorizer.get_feature_names_out()
   
    for topic_idx, keywords in topic_keywords.items():
        print(f"\nTopic {topic_idx + 1}:")
        # Format each word with its weight in parentheses
        formatted_words = [f"{word} ({weight:.2f})" 
                         for word, weight in keywords['word_weights']]
        print(", ".join(formatted_words)) 
    return lda_model, vectorizer, topic_distribution, df_field, topic_keywords

def model_topics(df, num_topics=10, num_words=5):
    
    print(f"Analyzing {len(df)} papers across all fields")
    
    if df.empty:
        print("No papers found for analysis")
        return None, None, None, None, None
        
    text_data = df['abstract'].fillna('')
    
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
    doc_term_matrix = vectorizer.fit_transform(text_data)
    
    lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    topic_distribution = lda_model.fit_transform(doc_term_matrix)
    topic_keywords = extract_topic_keywords(lda_model, vectorizer)
    
    for topic_idx, keywords in topic_keywords.items():
        print(f"\nTopic {topic_idx + 1}:")
        formatted_words = [f"{word} ({weight:.2f})" 
                         for word, weight in keywords['word_weights']]
        print(", ".join(formatted_words))
        
    return lda_model, vectorizer, topic_distribution, df, topic_keywords


# Generate topic name function
def generate_topic_name(text, keywords, client, model_type, credit_tracker):
    try:
        tokens = num_tokens_from_string(text + " " + " ".join(keywords), model_type)
        credit_tracker.update(tokens)
        response = client.chat.completions.create(
            model=model_type,
            messages=[
                {"role": "system", "content": "You are a helpful scientific assistant that generates concise topic expressions based on academic paper titles and keywords for a collection of papers."},
                {"role": "user", "content": f"""Based on the following titles and keywords obtained by LDA-analysis, provide a concise single word, bigram, or trigram that best describes the main topic these abstracts have in common. The expression should be specific and descriptive and feasible for categorization.

Title:
{text}

Keywords:
{', '.join(keywords)}

Concise topic expression:"""}
            ]
        )
        content = response.choices[0].message.content.strip()
        credit_tracker.update(num_tokens_from_string(content, model_type))
        return content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def string_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

def generate_topic_name_multiple(text, keywords, client, model_type, credit_tracker, initial_iterations=3, max_iterations=10, similarity_threshold=0.7):
    iterations = initial_iterations
    while iterations <= max_iterations:
        generated_names = []
        for _ in range(iterations):
            # Reuse the generate_topic_name function instead of duplicating code
            name = generate_topic_name(text, keywords, client, model_type, credit_tracker)
            if name:
                generated_names.append(name)
        
        # Check for dominant topic
        for i, name in enumerate(generated_names):
            similar_names = [other_name for j, other_name in enumerate(generated_names) 
                             if i != j and string_similarity(name, other_name) >= similarity_threshold]
            if len(similar_names) >= len(generated_names) // 2:
                return name  # Return the dominant topic name

        # If no dominant topic found, increase iterations
        iterations += 2
        print(f"No clear common topic name found. Increasing iterations to {iterations}.")

    # If max iterations reached without finding a dominant topic, return the most common name
    from collections import Counter
    return Counter(generated_names).most_common(1)[0][0]



def classify_papers(topic_distributions, df_field):
    """Classify papers based on topic distributions"""
    paper_classifications = []
    
    for idx, dist in enumerate(topic_distributions): #looping throug the papers one-by-one (index by index) and topics distribution 
        # Get primary and secondary topics (only primary is used in dominance ratio)
        top_2_topics = np.argsort(dist)[-2:][::-1] # extracting the to last [-2:] the order is reversed to highest first with the [::-1]
        
        # Calculate dominance ratios by :
        primary_score = dist[top_2_topics[0]] # 1) extracting the probability of the primary topic
        other_topics_sum = sum(dist) - primary_score # 2) calc. the sum of all the other topics (all topic-primary)
        dominance_ratio = primary_score / (other_topics_sum + 1e-10) #3) dividing the primary score by the sum (adding very small number to avoid dividing by zero if only one topic)
        
        # Storing the paper classifications
        paper_classifications.append({
            'paper_idx': idx,                   #storing the paper index
            'primary_topic': top_2_topics[0],   # the most probable topic
            'secondary_topic': top_2_topics[1], # second most probable topic
            'primary_score': primary_score,     # the probability of the most probable topic 
            'dominance_ratio': dominance_ratio  # the domincane ratio (eg. is the paper only about this, or also covering other topics?)
        })
    
    return paper_classifications

# Get top papers per topic per field

def get_top_papers(paper_classifications, df_field, n_top=5):
    top_papers = {}
    author_topic_stats = {}
    
    # Debug print
    #print(f"Total papers to analyze: {len(paper_classifications)}")
    
    for topic in set(p['primary_topic'] for p in paper_classifications): # creating a set of the unique primary topics in the datasett, e.g. if there are only three primary topics and all papers "belong" to one of them, the set will be {Primary topic 1, Primary topic 2, Primary topic 3}
        print(f"\nProcessing topic {topic}")
        topic_papers = [p for p in paper_classifications if p['primary_topic'] == topic]  # Get all the papers for the current topic
        print(f"Papers for topic {topic}: {len(topic_papers)}")
        topic_papers.sort(key=lambda x: x['dominance_ratio'], reverse=True) # Sort by dominance ratio, reverse=True to get the highest ratios at the top.
        top_papers[topic] = []
        
        # Get top n papers where n is the numebr spesified as top_n when calling the functino
        for p in topic_papers[:n_top]: #loop through the top_n number of papers
            paper_idx = p['paper_idx']
            authors = df_field.iloc[paper_idx]['authors']
            
            # Debug print
            #print(f"\nPaper index: {paper_idx}")
            #print(f"Authors data type: {type(authors)}")
            #print(f"Authors content: {authors}")
            
            # Check if authors is string (might be stored as JSON string)
            if isinstance(authors, str):
                try:
                    authors = ast.literal_eval(authors)
                except (ValueError, SyntaxError):
                    print(f"Failed to parse authors string: {authors}")
                    authors = []
            
            if isinstance(authors, list):
                author_list = []
                for author in authors:
                    # Debug print
                    print(f"Processing author: {author}")
                    if isinstance(author, dict):
                        author_list.append({
                            'name': author.get('name', 'Unknown'),
                            'id': author.get('authorId', 'Unknown')
                        })
                    else:
                        print(f"Unexpected author format: {author}")
            else:
                print(f"Unexpected authors format: {authors}")
                author_list = []
            
            # Debug print
            #print(f"Processed author list: {author_list}")
            
            top_papers[topic].append({
                'paperId': df_field.iloc[paper_idx]['paperId'],
                'title': df_field.iloc[paper_idx]['title'],
                'abstract': df_field.iloc[paper_idx]['abstract'],
                'authors': author_list,
                'score': float(p['primary_score']),
                'dominance_ratio': float(p['dominance_ratio'])
            })
            
            for author in author_list:
                author_id = author['id']
                if author_id not in author_topic_stats:
                    author_topic_stats[author_id] = {
                        'name': author['name'],
                        'topics': {},
                        'total_papers': 0,
                        'top_papers': 0
                    }
                
                if topic not in author_topic_stats[author_id]['topics']:
                    author_topic_stats[author_id]['topics'][topic] = {
                        'paper_count': 0,
                        'avg_dominance': 0,
                        'top_papers': []
                    }
                
                author_stats = author_topic_stats[author_id]['topics'][topic]
                author_stats['paper_count'] += 1
                author_stats['avg_dominance'] = (
                    (author_stats['avg_dominance'] * (author_stats['paper_count'] - 1) + 
                     float(p['dominance_ratio'])) / author_stats['paper_count']
                )
                author_stats['top_papers'].append({
                    'title': df_field.iloc[paper_idx]['title'],
                    'dominance_ratio': float(p['dominance_ratio'])
                })
                
                author_topic_stats[author_id]['total_papers'] += 1
                author_topic_stats[author_id]['top_papers'] += 1
    
      # Convert topic numbers to regular integers for JSON serialization
    author_topic_stats_clean = {}
    for author_id, stats in author_topic_stats.items():
        author_topic_stats_clean[author_id] = stats.copy()
        author_topic_stats_clean[author_id]['topics'] = {
            int(topic): topic_stats 
            for topic, topic_stats in stats['topics'].items()
        }
    
    # Debug print with cleaned data
    #print(f"\nFinal author_topic_stats: {json.dumps(author_topic_stats_clean, indent=2)}")
    
    return top_papers, author_topic_stats


def print_author_analysis(author_topic_stats, min_papers=2):
    """Print detailed author analysis"""
    print("\nAuthor Analysis:")
    for author_id, stats in author_topic_stats.items():
        if stats['total_papers'] >= min_papers:
            print(f"\nAuthor: {stats['name']}")
            print(f"Total papers in top lists: {stats['total_papers']}")
            print("Topics:")
            for topic, topic_stats in stats['topics'].items():
                print(f"\nTopic {topic}:")
                print(f"  Paper count: {topic_stats['paper_count']}")
                print(f"  Average dominance ratio: {topic_stats['avg_dominance']:.4f}")
                print("  Top papers:")
                for paper in topic_stats['top_papers']:
                    print(f"    - {paper['title']} (dominance: {paper['dominance_ratio']:.4f})")


In [9]:
""" needs fixing"""

def analyze_specific_topic(df, topic, num_subtopics=5, n_top=5):
    # Filter the dataframe for the specific topic
    df_topic = df[df['Primary_Topic'] == topic].copy()
    
    print(f"Analyzing topic: {topic}")
    print(f"Number of papers: {len(df_topic)}")
    
    if len(df_topic) < 10:  # Adjust this threshold as needed
        print("Not enough papers for meaningful subtopic analysis.")
        return None, None, None
    
    # Ensure index is unique
    df_topic = df_topic.reset_index(drop=True)
    
    # Prepare the text data
    text_data = df_topic['abstract'].fillna('')
    
    # Create document-term matrix
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
    doc_term_matrix = vectorizer.fit_transform(text_data)
    
    # Create and fit LDA model for subtopics
    lda_model = LatentDirichletAllocation(n_components=num_subtopics, random_state=42)
    subtopic_distributions = lda_model.fit_transform(doc_term_matrix)
    
    # Get top words for each subtopic
    feature_names = vectorizer.get_feature_names_out()
    for idx, subtopic in enumerate(lda_model.components_):
        top_words = [feature_names[i] for i in subtopic.argsort()[:-10 - 1:-1]]
        print(f"Subtopic {idx + 1}: {', '.join(top_words)}")
    
    # Get top papers for each subtopic
    top_papers = {}
    for subtopic in range(num_subtopics):
        subtopic_scores = subtopic_distributions[:, subtopic]
        other_subtopics_sum = subtopic_distributions.sum(axis=1) - subtopic_scores
        subtopic_dominance = subtopic_scores / (other_subtopics_sum + 1e-10)
        top_indices = np.argsort(subtopic_dominance)[-n_top:][::-1]
        
        top_papers[subtopic] = [
            {
                'title': df_topic.iloc[i]['title'],
                'abstract': df_topic.iloc[i]['abstract'],
                'score': float(subtopic_scores[i]),
                'dominance_ratio': float(subtopic_dominance[i])
            }
            for i in top_indices
        ]
    # Generate subtopic names using the same method as before - Skipped until copyright breach issue is resolved
    """
    subtopic_names = {}
    for subtopic, papers in top_papers.items():
        abstracts = "\n\n".join([paper['abstract'] for paper in papers])
        keywords = [word for paper in papers for word in paper['abstract'].split()[:10]]
        subtopic_name = generate_topic_name(abstracts, keywords)
        subtopic_names[subtopic] = subtopic_name
    """
    #Add subtopic scores and names to the dataframe
    for i in range(num_subtopics):
        df_topic[f'Subtopic_{i+1}_Score'] = subtopic_distributions[:, i]
        df_topic['Primary_Subtopic'] = df_topic[[f'Subtopic_{i+1}_Score' for i in range(num_subtopics)]].idxmax(axis=1)
   # df_topic['Primary_Subtopic_Name'] = df_topic['Primary_Subtopic'].map(lambda x: subtopic_names[int(x.split('_')[1]) - 1])
    
    # Update the main dataframe with the new subtopic information
        df_update = df.copy()
        df_update.loc[df_topic.index, [f'Subtopic_{i+1}_Score' for i in range(num_subtopics)]] = df_topic[[f'Subtopic_{i+1}_Score' for i in range(num_subtopics)]]
        df_update.loc[df_topic.index, 'Primary_Subtopic'] = df_topic['Primary_Subtopic']
  # df_update.loc[df_topic.index, 'Primary_Subtopic_Name'] = df_topic['Primary_Subtopic_Name']
    
    return df_update, top_papers#, subtopic_names

In [40]:

filename='semantic_scholar_2025_02_14_reliability_resilience_power_systems_results.csv' # if the search key words are changed you need to update the filename here. should be automatic...
filepath=os.path.join(SAVE_DIR,filename)
df=pd.read_csv(filepath,sep=";")
df['text'] = df['title'].fillna('') + ' ' + df['abstract'].fillna('')

In [41]:
# Processing to convert title and abstract to text field.
print("Preprocessing text data...")
search_keywords=['reliability', 'resilience', 'power system', 'capacity utilization'] # should be automitically retrieved from search script, but for the time beeing its manually input here...
df['processed_text'] = df['text'].apply(lambda x:preprocess_text(x,search_keywords=search_keywords))
print("Text preprocessing completed.")

Preprocessing text data...
Text preprocessing completed.


In [50]:

# preparing for analysis (need to fill the empty fieldsOfStudy with ''):
valid_fields= ['Computer Science', 'Economics', 'Engineering', 'Physics', 'Mathematics', 'Medicine', 'Business', 'Environmental Science', 'Chemistry', 'Materials Science', 'Geography', 'Biology', 'Geology', 'Political Science', 'Psychology', 'Com']
df['fieldsOfStudy'] = df['fieldsOfStudy'].apply(clean_fields_of_study)

In [15]:
# simple analysis before topic modelling
# Usage
extract_keywords_and_ngrams(df)

Results saved to Saved_files\term_frequencies_2025_02_28.json


In [46]:
# new better version of the analyzis:
"""
def analyze_papers_with_topic_names(df, fields_to_analyze,n_papers=5, output_suffix="_analyzed_results"):
    # Initialize OpenAI client
    client, model_type = initialize_openai()
    credit_tracker = CreditTracker()
    
    # Create a copy of the original dataframe to store all results
    df_analyzed = df.copy()
    all_author_stats = {}
    all_topic_names = {}
    
    for field in fields_to_analyze:
        print(f"\nAnalyzing field: {field}")
        
        # Step 1: Model topics
        lda_model, vectorizer, topic_distributions, df_field, topic_keywords = model_topics(
            df, field, num_topics=10)
        
        if lda_model is None:
            continue
            
        # Step 2: Classify papers and add to main dataframe
        paper_classifications = classify_papers(topic_distributions, df_field)
        
        # Add classifications to the analyzed dataframe
        for p in paper_classifications:
            idx = df_field.index[p['paper_idx']]
            df_analyzed.loc[idx, f'{field}_Primary_Topic'] = p['primary_topic']
            df_analyzed.loc[idx, f'{field}_Secondary_Topic'] = p['secondary_topic']
            df_analyzed.loc[idx, f'{field}_Primary_Score'] = p['primary_score']
            df_analyzed.loc[idx, f'{field}_Dominance_Ratio'] = p['dominance_ratio']
        
        # Step 3: Get top papers and author stats
        top_papers, author_stats = get_top_papers(paper_classifications, df_field, n_top=n_papers)
        
        # Step 4: Generate topic names
        field_topic_names = {}
        for topic_idx, papers in top_papers.items():
            # Combine abstracts and extract keywords
            input_text = "\n\n".join([str(paper.get('title', '')) for paper in papers])
            keywords = topic_keywords[topic_idx]['top_words']
            
            # Generate topic name
            topic_name = generate_topic_name_multiple(input_text, keywords, client, model_type, credit_tracker)
            
            if topic_name:
                field_topic_names[topic_idx] = topic_name
                # Add topic name to dataframe
                topic_column = f'{field}_Topic_{topic_idx}_Name'
                df_analyzed[topic_column] = topic_name
                
                # Update primary topic name for papers with this primary topic
                primary_topic_mask = df_analyzed[f'{field}_Primary_Topic'] == topic_idx
                df_analyzed.loc[primary_topic_mask, f'{field}_Primary_Topic_Name'] = topic_name
            
            # Print topic name and top papers
            print(f"\nTopic {topic_idx + 1}: {topic_name if topic_name else 'Unnamed'}")
            print(f"Keywords: {', '.join(topic_keywords[topic_idx]['top_words'])}")
            for paper in papers:
                print(f"- {paper['title']} (dominance: {paper['dominance_ratio']:.4f})")
        
        # Store results
        all_author_stats[field] = author_stats
        all_topic_names[field] = field_topic_names
        
        # Add topic keywords to dataframe metadata
        df_analyzed.attrs[f'{field}_topic_keywords'] = topic_keywords
        df_analyzed.attrs[f'{field}_topic_names'] = field_topic_names
    
    # Save results
    current_date = datetime.now().strftime("%Y_%m_%d")
    output_filename = os.path.join(SAVE_DIR,f"semantic_scholar_{current_date}{output_suffix}.csv")
    
    # Save main results
    df_analyzed.to_csv(output_filename, sep=';', encoding='utf-8', 
                      quoting=csv.QUOTE_NONNUMERIC, escapechar='\\')
    
    # Save author statistics with topic frequencies
    author_filename = os.path.join(SAVE_DIR,f"semantic_scholar_{current_date}_author_analysis.csv")
    save_author_analysis(all_author_stats, author_filename)
    
    # Save topic names
    converted_topic_names = {}
    for field, topics in all_topic_names.items():
        converted_topic_names[field] = {
        int(topic_idx): name 
        for topic_idx, name in topics.items()
        }

    topic_names_filename = os.path.join(SAVE_DIR,f"semantic_scholar_{current_date}_topic_names.json")
    with open(topic_names_filename, 'w', encoding='utf-8') as f:
        json.dump(all_topic_names, f, ensure_ascii=False, indent=2)
    
    # Print credit usage
    print("\nAPI Usage Statistics:")
    stats = credit_tracker.get_stats()
    print(f"Total tokens: {stats['total_tokens']}")
    print(f"Estimated cost: ${stats['total_cost']}")
    
    return df_analyzed, all_author_stats, all_topic_names
"""
def analyze_papers_with_topic_names(df, fields_to_analyze=None, n_papers=5, output_suffix="_analyzed_results"):
    """
    Analyze papers with topic naming
    
    Parameters:
    df - DataFrame containing papers
    fields_to_analyze - List of fields to filter papers by (if None, use all papers)
    n_papers - Number of top papers to extract per topic
    output_suffix - Suffix for output files
    """
    # Initialize OpenAI client
    client, model_type = initialize_openai()
    credit_tracker = CreditTracker()
    
    # Create a copy of the original dataframe to store all results
    df_analyzed = df.copy()
    all_author_stats = {}
    all_topic_names = {}
    
    # Filter dataset by fields if specified
    if fields_to_analyze:
        df_filtered = df[df['fieldsOfStudy'].apply(lambda x: any(field in x for field in fields_to_analyze))]
        print(f"Filtered to {len(df_filtered)} papers from fields: {', '.join(fields_to_analyze)}")
    else:
        df_filtered = df
        print(f"Using all {len(df_filtered)} papers for analysis")
    
    if df_filtered.empty:
        print("No papers found after filtering")
        return None, None, None
    
    # Step 1: Model topics on the filtered dataset
    lda_model, vectorizer, topic_distributions, df_field, topic_keywords = model_topics(
        df_filtered, num_topics=10)
    
    if lda_model is None:
        return None, None, None
        
    # Step 2: Classify papers and add to main dataframe
    paper_classifications = classify_papers(topic_distributions, df_field)
    
    # Step 3: Generate topic names first, so we can use them in column names
    top_papers, author_stats = get_top_papers(paper_classifications, df_field, n_top=n_papers)
    
    # Generate topic names
    topic_names = {}
    for topic_idx, papers in top_papers.items():
        # Combine titles and extract keywords
        input_text = "\n\n".join([str(paper.get('title', '')) for paper in papers])
        keywords = topic_keywords[topic_idx]['top_words']
        
        # Generate topic name
        topic_name = generate_topic_name_multiple(input_text, keywords, client, model_type, credit_tracker)
        
        if topic_name:
            topic_names[topic_idx] = topic_name
            
        # Print topic name and top papers
        print(f"\nTopic {topic_idx + 1}: {topic_name if topic_name else 'Unnamed'}")
        print(f"Keywords: {', '.join(topic_keywords[topic_idx]['top_words'])}")
        for paper in papers:
            print(f"- {paper['title']} (dominance: {paper['dominance_ratio']:.4f})")
    
    # Step 4: Add classifications to the analyzed dataframe using topic names
    for p in paper_classifications:
        idx = df_field.index[p['paper_idx']]
        primary_topic_idx = p['primary_topic']
        secondary_topic_idx = p['secondary_topic']
        
        # Store topic indices for reference
        df_analyzed.loc[idx, 'Primary_Topic_Index'] = primary_topic_idx
        df_analyzed.loc[idx, 'Secondary_Topic_Index'] = secondary_topic_idx
        
        # Store topic names as primary columns
        primary_name = topic_names.get(primary_topic_idx, f"Topic_{primary_topic_idx}")
        secondary_name = topic_names.get(secondary_topic_idx, f"Topic_{secondary_topic_idx}")
        
        df_analyzed.loc[idx, 'Primary_Topic'] = primary_name
        df_analyzed.loc[idx, 'Secondary_Topic'] = secondary_name
        df_analyzed.loc[idx, 'Primary_Score'] = p['primary_score']
        df_analyzed.loc[idx, 'Dominance_Ratio'] = p['dominance_ratio']
    
    # Store results
    all_author_stats['all'] = author_stats
    all_topic_names['all'] = topic_names
    
    # Add topic keywords to dataframe metadata
    df_analyzed.attrs['topic_keywords'] = topic_keywords
    df_analyzed.attrs['topic_names'] = topic_names
    
    # Save results
    current_date = datetime.now().strftime("%Y_%m_%d")
    output_filename = os.path.join(SAVE_DIR, f"semantic_scholar_{current_date}{output_suffix}.csv")
    
    # Save main results
    df_analyzed.to_csv(output_filename, sep=';', encoding='utf-8', 
                      quoting=csv.QUOTE_NONNUMERIC, escapechar='\\')
    
    # Save author statistics with topic frequencies
    author_filename = os.path.join(SAVE_DIR, f"semantic_scholar_{current_date}_author_analysis.csv")
    save_author_analysis(all_author_stats, author_filename)
    
    # Save topic names
    converted_topic_names = {}
    for field, topics in all_topic_names.items():
        converted_topic_names[field] = {
            int(topic_idx): name 
            for topic_idx, name in topics.items()
        }

    topic_names_filename = os.path.join(SAVE_DIR, f"semantic_scholar_{current_date}_topic_names.json")
    with open(topic_names_filename, 'w', encoding='utf-8') as f:
        json.dump(converted_topic_names, f, ensure_ascii=False, indent=2)
    
    # Print credit usage
    print("\nAPI Usage Statistics:")
    stats = credit_tracker.get_stats()
    print(f"Total tokens: {stats['total_tokens']}")
    print(f"Estimated cost: ${stats['total_cost']}")
    
    return df


def save_author_analysis(all_author_stats, filename):
    """Save detailed author analysis including topic frequencies"""
    author_rows = []
    
    for field, author_stats in all_author_stats.items():
        for author_id, stats in author_stats.items():
            for topic, topic_stats in stats['topics'].items():
                author_rows.append({
                    'Field': field,
                    'Author_ID': author_id,
                    'Author_Name': stats['name'],
                    'Topic': topic,
                    'Paper_Count': topic_stats['paper_count'],
                    'Avg_Dominance': topic_stats['avg_dominance'],
                    'Total_Papers': stats['total_papers']
                })
    
    author_df = pd.DataFrame(author_rows)
    author_df.to_csv(filename, sep=';', encoding='utf-8', 
                    quoting=csv.QUOTE_NONNUMERIC, index=False)

In [52]:
# Analyze papers with topic naming
fields_to_analyze = ['Computer Science', 'Economics', 'Engineering', 'Physics', 'Mathematics']
df_analyzed = analyze_papers_with_topic_names(df,fields_to_analyze=fields_to_analyze, n_papers=10)


Filtered to 16939 papers from fields: Computer Science, Economics, Engineering, Physics, Mathematics
Analyzing 16939 papers across all fields

Topic 1:
power (8399.84), high (3621.12), voltage (3567.04), control (3530.43), current (3223.15), dc (2746.68), circuit (1964.44), converter (1857.09), reliability (1823.41), phase (1489.17)

Topic 2:
data (5036.40), systems (3021.98), performance (2468.24), based (2304.72), power (1967.72), reliability (1920.28), computing (1891.35), time (1872.24), memory (1780.22), applications (1666.71)

Topic 3:
energy (2607.99), systems (2480.62), power (2312.30), gas (1783.11), water (1762.72), heat (1735.19), plant (1578.83), development (1502.95), design (1453.81), technology (1452.37)

Topic 4:
power (7846.12), network (3354.04), distribution (3292.57), reliability (2673.37), transmission (2565.92), grid (2326.40), fault (2175.20), systems (1710.60), line (1541.14), paper (1528.06)

Topic 5:
battery (2894.33), charging (1602.86), vehicle (1474.09), el

In [ ]:
# Define valid fields and clean fields of study
valid_fields = ['Computer Science', 'Economics', 'Engineering', 'Physics', 'Mathematics', 'Medicine', 'Business', 'Environmental Science', 'Chemistry', 'Materials Science', 'Geography', 'Biology', 'Geology', 'Political Science', 'Psychology', 'Com']
df['fieldsOfStudy'] = df['fieldsOfStudy'].apply(clean_fields_of_study)
# Filter out papers with excluded fields
exclude_fields = ['Chemistry', 'Materials Science', 'Geography', 'Biology', 'Geology', 'Medicine', 'Political Science', 'Psychology', 'Com']
df_filtered = df[df['fieldsOfStudy'].apply(lambda x: not set(x).issubset(set(exclude_fields)))]
# Get unique fields of study
unique_fields = set([field for fields in df_filtered['fieldsOfStudy'] for field in fields if field not in exclude_fields])
